In [ ]:
!pip install llama-index

In [4]:
import os
import openai
from IPython.display import Markdown, display

Let's create DB with a table and insert some

In [19]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [20]:
# creating SQL table
table_name = "company_financials"
company_financials = Table(
    table_name,
    metadata_obj,
    Column("year", Integer, primary_key=True),
    Column("revenue", Integer),
    Column("profit", Integer)
)
metadata_obj.create_all(engine)

In [21]:
from sqlalchemy import insert

rows = [
    {"year": 2023, "revenue": 10000000, "profit": 1000000},
    {"year": 2022, "revenue": 1000000, "profit": 100000},
    {"year": 2021, "revenue": 100000, "profit": 20000},
]
for row in rows:
    stmt = insert(company_financials).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [22]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["company_financials"])

For this test, let's use GPT-3.5

In [23]:
from llama_index import SQLDatabase, ServiceContext
from llama_index.llms import OpenAI

os.environ["OPENAI_API_KEY"] = "..."
openai.api_key = os.environ["OPENAI_API_KEY"]
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)


In [25]:
# print current table
stmt = select(
    company_financials.c.year,
    company_financials.c.revenue,
    company_financials.c.profit,
).select_from(company_financials)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[(2021, 100000, 20000), (2022, 1000000, 100000), (2023, 10000000, 1000000)]


Asking LLM questions about data

In [28]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["company_financials"],
)
query_str = "Which year has the highest reveneu?"
response = query_engine.query(query_str)
print(response)

The year with the highest revenue is 2023, with a revenue of $10,000,000.


In [29]:
query_str = "What is your analysis about company financials?"
response = query_engine.query(query_str)
print(response)

Based on the analysis of the company financials, the revenue and profit have been steadily increasing over the years. In 2021, the company had a revenue of $100,000 and a profit of $20,000. In 2022, the revenue increased to $1,000,000 and the profit increased to $100,000. In 2023, the revenue further increased to $10,000,000 and the profit increased to $1,000,000. This indicates a positive trend in the company's financial performance.


In [30]:
query_str = "Do you have any prediction for next year based on this info?"
response = query_engine.query(query_str)
print(response)

Based on the information provided, the company's financials for the most recent year available (2023) show a revenue of $10,000,000 and a profit of $1,000,000. However, without further data or analysis, it is not possible to make accurate predictions for the next year.
